In [ ]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy
# import json
import pandas as pd

In [ ]:
dataset = pd.read_excel('labeled_dataset_final.xlsx')
dataset = dataset.dropna()
dataset

,drone_model,dataset,controller,message,tokenized,label
0,DJI_Inspire_1,df010,mobile_iOS,Taking Off.,"['Taking', 'Off', '.']","['B-ACTION', 'I-ACTION', 'O']"
1,DJI_Inspire_1,df010,mobile_iOS,Taking Off.,"['Taking', 'Off', '.']","['B-ACTION', 'I-ACTION', 'O']"
2,DJI_Inspire_1,df010,mobile_iOS,Taking Off.,"['Taking', 'Off', '.']","['B-ACTION', 'I-ACTION', 'O']"
3,DJI_Inspire_1,df010,mobile_iOS,Taking Off.,"['Taking', 'Off', '.']","['B-ACTION', 'I-ACTION', 'O']"
4,DJI_Inspire_1,df010,mobile_iOS,Home Point Recorded. RTH Altitude : 30m.,"['Home', 'Point', 'Recorded', '.', 'RTH', 'Alt...","['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-PA..."
...,...,...,...,...,...,...
1848,DJI_Spark,df008,mobile_iOS_backup,Obstacle Avoidance will be disabled in landing.,"['Obstacle', 'Avoidance', 'will', 'be', 'disab...","['B-FUNCTION', 'I-FUNCTION', 'O', 'O', 'O', 'O..."
1849,DJI_Spark,df008,mobile_iOS_backup,The remaining battery is only enough for RTH. ...,"['The', 'remaining', 'battery', 'is', 'only', ...","['O', 'O', 'B-COMPONENT', 'O', 'O', 'O', 'O', ..."
1850,DJI_Spark,df008,mobile_iOS,Home Point Recorded. RTH Altitude : 30m.,"['Home', 'Point', 'Recorded', '.', 'RTH', 'Alt...","['B-PARAMETER', 'I-PARAMETER', 'O', 'O', 'B-PA..."
1851,DJI_Spark,df008,mobile_iOS,Fly with caution and ensure the aircraft remai...,"['Fly', 'with', 'caution', 'and', 'ensure', 't...","['O', 'O', 'O', 'O', 'O', 'O', 'B-COMPONENT', ..."


In [ ]:
dataset.shape

(1850, 6)

In [ ]:
print(dataset.iloc[8, [4,5]])

tokenized    ['Your', 'palm', 'is', 'too', 'far', 'away', '...
label        ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-CO...
Name: 8, dtype: object


In [ ]:
def toList(sourceString):
    first_tokenized = sourceString
    first_tokenized = first_tokenized[1:-1].split(',') # Remove the bracket [] and split by comma
#     first_tokenized = first_tokenized.replace("[", "").replace("]", "").split(',')
    first_tokenized = [word.strip() for word in first_tokenized] # Remove white space
    first_tokenized = [word[1:-1] for word in first_tokenized] # Remove quote ''
#     print(first_tokenized)
#     print(type(first_tokenized))
    return first_tokenized
# print(toList(dataset.iloc[156, 4]))

In [ ]:
toList(dataset.iloc[8, 4])

['Your',
 'palm',
 'is',
 'too',
 'far',
 'away',
 'from',
 'the',
 'aircraft',
 '.',
 'Please',
 'step',
 'closer',
 '.']

In [ ]:
def buildDataset(dataset, start_idx):
  data_list = []
  data_diff = []
  for row in range(0,dataset.shape[0]):
      drone_model = dataset.iloc[row, 0]
      df_code = dataset.iloc[row, 1]
      controller = dataset.iloc[row, 2]
      message = dataset.iloc[row, 3]
      tokenized = toList(dataset.iloc[row, 4])
      label = toList(dataset.iloc[row, 5])
  #     print(type(tokenized))
  #     print(type(label))
  #     print('Row: %d' % int(row+1))
  #     print('token: %d, label: %d' % (len(tokenized), len(label)))
      if(len(tokenized) == len(label)):
          for token in range (0, len(tokenized)):
              tag = label[token]
              word = tokenized[token]
              data_list.append([drone_model, df_code, controller, (start_idx+1), word, tag])
      else:
  #         print("%d: The lengh is not the same" % row)
          token_len = len(tokenized)
          label_len = len(label)
          data_diff.append([drone_model, df_code, controller, message, tokenized, token_len, label, label_len])
      start_idx += 1
  return data_list, data_diff

In [ ]:
cleaned_list, broken_list = buildDataset(dataset, 0)

In [ ]:
cleaned_data = pd.DataFrame(cleaned_list, columns=['drone_model', 'dataset', 'controller', 'sentence_id', 'words', 'labels'])
broken_data = pd.DataFrame(broken_list, columns=['drone_model', 'dataset', 'controller', 'message', 'tokenized', 'token_len', 'label', 'label_len'])
cleaned_data

,drone_model,dataset,controller,sentence_id,words,labels
0,DJI_Inspire_1,df010,mobile_iOS,1,Taking,B-ACTION
1,DJI_Inspire_1,df010,mobile_iOS,1,Off,I-ACTION
2,DJI_Inspire_1,df010,mobile_iOS,1,.,O
3,DJI_Inspire_1,df010,mobile_iOS,2,Taking,B-ACTION
4,DJI_Inspire_1,df010,mobile_iOS,2,Off,I-ACTION
...,...,...,...,...,...,...
12404,DJI_Spark,df008,mobile_iOS,1850,home,O
12405,DJI_Spark,df008,mobile_iOS,1850,shifts,O
12406,DJI_Spark,df008,mobile_iOS,1850,to,O
12407,DJI_Spark,df008,mobile_iOS,1850,landing,O


In [ ]:
sentence_number_per_df = cleaned_data.groupby(['drone_model', 'dataset'], as_index=False)['sentence_id'].agg(lambda x: len(set(x)))
sentence_number_per_df

,drone_model,dataset,sentence_id
0,DJI_Inspire_1,df010,58
1,DJI_Inspire_1,df011,40
2,DJI_Inspire_2,df025,83
3,DJI_Inspire_2,df026,108
4,DJI_Inspire_2,df027,249
5,DJI_Matrice_210,DF059,34
6,DJI_Matrice_210,DF060,49
7,DJI_Matrice_600,df034,95
8,DJI_Mavic_2,df067,26
9,DJI_Mavic_2,df068,114


In [ ]:
broken_data.shape
broken_data

,drone_model,dataset,controller,message,tokenized,token_len,label,label_len
0,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
1,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
2,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
3,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
4,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
5,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
6,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
7,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18
8,DJI_Inspire_2,df026,mobile_android_physical,"Home Point Recorded, Return-to-Home Altitude :...","[Home, Point, Recorded, , , Return, -, to, -, ...",13,"[B-PARAMETER, I-PARAMETER, O, O, B-PARAMETER, ...",12
9,DJI_Matrice_600,df034,mobile_android_logical,"Aircraft is in Attitude mode, so that it will ...","[Aircraft, is, in, Attitude, mode, , , so, tha...",19,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...",18


In [ ]:
broken_data.to_csv('broken_data.csv')
# Fix the broken data using Ms.Excel (manually)

In [ ]:
# Load the fixed broken data to merge with the previous unbroken data.
broken_fixed = pd.read_csv('broken_fixed.csv')
broken_fixed.drop(['token_len', 'label_len', 'Unnamed: 0'], inplace=True, axis=1)
broken_fixed

,drone_model,dataset,controller,message,tokenized,label
0,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,DJI_Inspire_1,df010,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
5,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
6,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
7,DJI_Inspire_1,df011,mobile_android,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
8,DJI_Inspire_2,df026,mobile_android_physical,"Home Point Recorded, Return-to-Home Altitude :...","['Home', 'Point', 'Recorded', 'Return', '-', '...","['B-PARAMETER', 'I-PARAMETER', 'O', 'B-PARAMET..."
9,DJI_Matrice_600,df034,mobile_android_logical,"Aircraft is in Attitude mode, so that it will ...","['Aircraft', 'is', 'in', 'Attitude', 'mode', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [ ]:
broken_fixed_list, _ = buildDataset(broken_fixed, 1850)

In [ ]:
broken_fixed_pd = pd.DataFrame(broken_fixed_list, columns=['drone_model', 'dataset', 'controller', 'sentence_id', 'words', 'labels'])
broken_fixed_pd

,drone_model,dataset,controller,sentence_id,words,labels
0,DJI_Inspire_1,df010,mobile_android,1851,Aircraft,O
1,DJI_Inspire_1,df010,mobile_android,1851,is,O
2,DJI_Inspire_1,df010,mobile_android,1851,in,O
3,DJI_Inspire_1,df010,mobile_android,1851,Attitude,O
4,DJI_Inspire_1,df010,mobile_android,1851,mode,O
...,...,...,...,...,...,...
943,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,Please,O
944,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,fly,O
945,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,with,O
946,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,caution,O


In [ ]:
sentence_broken = broken_fixed_pd.sentence_id.unique()
len(sentence_broken)

59

In [ ]:
# Merge the unbroken with fixed broken data
merged_data = pd.concat([cleaned_data, broken_fixed_pd], ignore_index=True)
merged_data

,drone_model,dataset,controller,sentence_id,words,labels
0,DJI_Inspire_1,df010,mobile_iOS,1,Taking,B-ACTION
1,DJI_Inspire_1,df010,mobile_iOS,1,Off,I-ACTION
2,DJI_Inspire_1,df010,mobile_iOS,1,.,O
3,DJI_Inspire_1,df010,mobile_iOS,2,Taking,B-ACTION
4,DJI_Inspire_1,df010,mobile_iOS,2,Off,I-ACTION
...,...,...,...,...,...,...
13352,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,Please,O
13353,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,fly,O
13354,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,with,O
13355,DJI_Phantom_4_Pro_V2,df063,mobile_android_logical,1909,caution,O


In [ ]:
sentence_number_per_df = merged_data.groupby(['drone_model', 'dataset'], as_index=False)['sentence_id'].agg(lambda x: len(set(x)))
sentence_number_per_df

,drone_model,dataset,sentence_id
0,DJI_Inspire_1,df010,62
1,DJI_Inspire_1,df011,44
2,DJI_Inspire_2,df025,83
3,DJI_Inspire_2,df026,109
4,DJI_Inspire_2,df027,249
5,DJI_Matrice_210,DF059,34
6,DJI_Matrice_210,DF060,49
7,DJI_Matrice_600,df034,96
8,DJI_Mavic_2,df067,31
9,DJI_Mavic_2,df068,117


In [ ]:
inspire_1 = merged_data[merged_data['drone_model'] == 'DJI_Inspire_1']
inspire_2 = merged_data[merged_data['drone_model'] == 'DJI_Inspire_2']
matrice_210 = merged_data[merged_data['drone_model'] == 'DJI_Matrice_210']
matrice_600 = merged_data[merged_data['drone_model'] == 'DJI_Matrice_600']
mavic_2 = merged_data[merged_data['drone_model'] == 'DJI_Mavic_2']
mavic_air = merged_data[merged_data['drone_model'] == 'DJI_Mavic_Air']
mavic_pro = merged_data[merged_data['drone_model'] == 'DJI_Mavic_Pro']
phantom_3 = merged_data[merged_data['drone_model'] == 'DJI_Phantom_3']
phantom_4 = merged_data[merged_data['drone_model'] == 'DJI_Phantom_4']
phantom_4_pro = merged_data[merged_data['drone_model'] == 'DJI_Phantom_4_Pro_V2']
phantom_s1000 = merged_data[merged_data['drone_model'] == 'DJI_S1000+']
phantom_spark = merged_data[merged_data['drone_model'] == 'DJI_Spark']

In [ ]:
phantom_s1000

,drone_model,dataset,controller,sentence_id,words,labels
11934,DJI_S1000+,df043,mobile_iOS_backup,1800,Strong,B-ISSUE
11935,DJI_S1000+,df043,mobile_iOS_backup,1800,wireless,I-ISSUE
11936,DJI_S1000+,df043,mobile_iOS_backup,1800,interference,I-ISSUE
11937,DJI_S1000+,df043,mobile_iOS_backup,1800,.,O
11938,DJI_S1000+,df043,mobile_iOS_backup,1800,Please,O
...,...,...,...,...,...,...
12081,DJI_S1000+,df043,mobile_iOS_backup,1816,Please,O
12082,DJI_S1000+,df043,mobile_iOS_backup,1816,fly,O
12083,DJI_S1000+,df043,mobile_iOS_backup,1816,with,O
12084,DJI_S1000+,df043,mobile_iOS_backup,1816,caution,O


In [ ]:
phantom_spark.shape

(323, 6)

In [ ]:
# phantom_4_pro.tag.value_counts() [inspire_1, inspire_2, matrice_210, matrice_600, mavic_2, mavic_air, mavic_pro, phantom_3, phantom_s1000]

In [ ]:
phantom_4.labels.value_counts()

O              788
I-PARAMETER     59
B-PARAMETER     43
B-COMPONENT     39
I-ISSUE         37
B-ACTION        37
B-ISSUE         32
B-FUNCTION      24
I-FUNCTION      20
I-STATE         17
B-STATE         15
I-ACTION        10
I-COMPONENT      3
Name: labels, dtype: int64

In [ ]:
test_df = pd.concat([phantom_4_pro, phantom_4, phantom_spark, phantom_3])
test_df

,drone_model,dataset,controller,sentence_id,words,labels
10359,DJI_Phantom_4_Pro_V2,df061,mobile_android_physical,1627,Backward,B-FUNCTION
10360,DJI_Phantom_4_Pro_V2,df061,mobile_android_physical,1627,Obstacle,I-FUNCTION
10361,DJI_Phantom_4_Pro_V2,df061,mobile_android_physical,1627,Sensing,I-FUNCTION
10362,DJI_Phantom_4_Pro_V2,df061,mobile_android_physical,1627,is,O
10363,DJI_Phantom_4_Pro_V2,df061,mobile_android_physical,1627,not,O
...,...,...,...,...,...,...
12949,DJI_Phantom_3,df002,mobile_android_backup,1883,Please,O
12950,DJI_Phantom_3,df002,mobile_android_backup,1883,fly,O
12951,DJI_Phantom_3,df002,mobile_android_backup,1883,with,O
12952,DJI_Phantom_3,df002,mobile_android_backup,1883,caution,O


In [ ]:
test_df.labels.value_counts()

O              2949
I-PARAMETER     195
B-PARAMETER     153
I-ISSUE         143
B-COMPONENT     132
B-ISSUE         101
B-ACTION        100
I-FUNCTION       86
B-FUNCTION       77
B-STATE          71
I-STATE          48
I-ACTION         25
I-COMPONENT      15
Name: labels, dtype: int64

In [ ]:
sentence_test = test_df.sentence_id.unique()
len(sentence_test)

438

In [ ]:
train_df = pd.concat([inspire_1, inspire_2, matrice_210, matrice_600, mavic_2, mavic_air, mavic_pro, phantom_s1000])
train_df.shape

(9262, 6)

In [ ]:
unique_train = train_df.sentence_id.unique()
len(unique_train)

1395

In [ ]:
train_df.to_csv('droner_train_4.csv')
test_df.to_csv('droner_test_4.csv')